# Prepare Recipe data

In [1]:
import pandas as pd
import constants
import json
import numpy as np

In [2]:
constants.ROOT_DIR

PosixPath('/Users/shubham/Desktop/shubham/upliance')

In [3]:
# load data
with open(constants.ROOT_DIR / "recipe_category.json", 'r') as f:
    recipe_cat_dict = json.load(f)

with open(constants.ROOT_DIR / "recipe_ingredients.json", 'r') as f:
    recipe_ingredients_dict = json.load(f)

In [4]:
category_df = pd.json_normalize(recipe_cat_dict['data']['recipe_categories'])
category_df.head()

,category_name,is_active,id,rank,Recipe_Categories_Map
0,Homely Recipes,True,69,0,[{'Recipe_ID': '5c7c1c8a-582c-48f7-9739-bcca75...
1,North Indian,True,120,0,[{'Recipe_ID': '18d527d2-19ac-46f4-a0ff-8d68b2...
2,Healthy Recipes,True,119,0,"[{'Recipe_ID': 'bM2PYm6kCCo6EmuPNQ11'}, {'Reci..."
3,Fusion Food,True,118,0,[{'Recipe_ID': 'dfc05dd1-3466-42a5-ac4d-063603...
4,South Indian,True,117,0,[{'Recipe_ID': '5756b48b-dc97-44d0-b699-bad97d...


In [5]:
# Category Recipe map
category_recipe_map = (
    category_df
    # dropping Recommende for you category
    .loc[lambda df_: df_['category_name'] != 'Recommended For You']
    [['id', 'Recipe_Categories_Map']]
    .explode('Recipe_Categories_Map')
    .loc
    [lambda df_: ~df_['Recipe_Categories_Map'].isnull()]
    .assign(recipe_id = lambda df_: df_['Recipe_Categories_Map'].map(lambda x: x['Recipe_ID']))
    .drop(['Recipe_Categories_Map'], axis=1)
    .rename(columns={"id": "category_id"})
)
category_recipe_map.head()

,category_id,recipe_id
0,69,5c7c1c8a-582c-48f7-9739-bcca7511ac7b
0,69,98061e99-3fd6-486e-a332-1b2b42eff358
0,69,99c9143a-489e-4e2c-afdd-a6ae8975e760
0,69,3bf5afd4-4f93-4ade-8ceb-caea8c1faeee
0,69,b82a3f5e-2c77-4256-bb02-a60d1cde1754


In [6]:
# Category
category_df = (
    category_df
    .drop(['is_active', 'rank', 'Recipe_Categories_Map'], axis=1)
    # deleting recommended for you catefory
    .loc[lambda df_: df_['category_name'] != 'Recommended For You']
)
category_df.head()

,category_name,id
0,Homely Recipes,69
1,North Indian,120
2,Healthy Recipes,119
3,Fusion Food,118
4,South Indian,117


In [7]:
recipe_df = pd.json_normalize(recipe_ingredients_dict['data']['Recipes_E3'])
recipe_df.head()

,id,name,cuisine,hidden_tags,diet,description,meal_type,active,is_public,created_source,Recipe_Variations
0,54087d1b-6dab-403b-93eb-c33162f10aae,Zucchini Curry,[Indian],[],[Vegan],Zucchini curry is a vegetarian dish that featu...,[Main Course Gravy],True,True,recipeAdmin,[{'description': 'Zucchini curry is a vegetari...
1,b7f22d40-8a95-4877-8d50-0f493c61738a,Gujarati Kadhi,"[Indian, Gujarati]",[],[Veg],Gujarati kadhi is a sweet tangy yogurt-based d...,[Main Course Gravy],True,True,recipeAdmin,[{'description': 'Gujarati kadhi is a sweet ta...
2,a15df9da-6efd-4bf1-ba6e-0f5fa0fc6a4e,Sabudana Khichdi,[Indian],[],[Veg],Sabudana Khichdi is a popular Indian dish made...,[Breakfast],True,True,recipeAdmin,[{'description': 'Sabudana Khichdi is a popula...
3,f5e954c2-821a-4541-a3b0-d126baa02ac5,Beetroot Palya,"[Indian, South Indian]",[],[Vegan],"This recipe is prepared using beetroot, grated...",[Main Course Dry],True,True,recipeAdmin,[{'description': 'This recipe is prepared usin...
4,f7b53d60-5c24-4512-bba5-90af1f27f8ce,Jain Red Sauce Pasta,[Italian],[],[Veg],Red sauce pasta on autopilot. Lightly flavoure...,[One Pot Meal],True,True,recipeAdmin,[{'description': 'Red sauce pasta on autopilot...


In [8]:
# Recipe
recipe_df = (
    recipe_df
    .assign(Recipe_Variations= lambda df_: df_['Recipe_Variations'].map(lambda x: x[0]))
    .assign(
        image_url = lambda df_: df_['Recipe_Variations'].map(lambda x: x['image_url']),
        cooking_time = lambda df_: df_['Recipe_Variations'].map(lambda x: x['cooking_time']),
        prep_time = lambda df_: df_['Recipe_Variations'].map(lambda x: x['prep_time']),
        ingredients = lambda df_: df_['Recipe_Variations'].map(lambda x: x['ingredients'])
    )
    .drop(['hidden_tags', 'active', 'is_public', 'created_source', 'Recipe_Variations',], axis=1)
)
recipe_df.head()

,id,name,cuisine,diet,description,meal_type,image_url,cooking_time,prep_time,ingredients
0,54087d1b-6dab-403b-93eb-c33162f10aae,Zucchini Curry,[Indian],[Vegan],Zucchini curry is a vegetarian dish that featu...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,66.0,0.0,"[YZJypF7J7qoqJ5JcuzXs, jVF1elDY6pZ1zZfoWO9K, k..."
1,b7f22d40-8a95-4877-8d50-0f493c61738a,Gujarati Kadhi,"[Indian, Gujarati]",[Veg],Gujarati kadhi is a sweet tangy yogurt-based d...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,36.0,0.0,"[SSkQlAri5V5Ip377WXqQ, k01MJnr4IWnpM9vUG9Cp, 4..."
2,a15df9da-6efd-4bf1-ba6e-0f5fa0fc6a4e,Sabudana Khichdi,[Indian],[Veg],Sabudana Khichdi is a popular Indian dish made...,[Breakfast],https://res.cloudinary.com/dgakjrw8j/image/upl...,58.0,240.0,"[b8301341-efcd-425c-86f0-e282415f4a9b, OqJKyZC..."
3,f5e954c2-821a-4541-a3b0-d126baa02ac5,Beetroot Palya,"[Indian, South Indian]",[Vegan],"This recipe is prepared using beetroot, grated...",[Main Course Dry],https://res.cloudinary.com/dgakjrw8j/image/upl...,31.0,0.0,"[jVF1elDY6pZ1zZfoWO9K, 1Ky6vVG6LIdXgQ77KpUf, k..."
4,f7b53d60-5c24-4512-bba5-90af1f27f8ce,Jain Red Sauce Pasta,[Italian],[Veg],Red sauce pasta on autopilot. Lightly flavoure...,[One Pot Meal],https://res.cloudinary.com/dgakjrw8j/image/upl...,46.0,0.0,"[cJvpPYfSg5s1aEaEzPv8, 1Ky6vVG6LIdXgQ77KpUf, L..."


In [9]:
# Recipe Ingredients map
recipe_ingredients_df = (
    recipe_df[['id', 'ingredients']]
    .rename(columns={'id': "recipe_id", "ingredients": "ingredients_id"})
    .explode('ingredients_id')
)
recipe_ingredients_df.head()

,recipe_id,ingredients_id
0,54087d1b-6dab-403b-93eb-c33162f10aae,YZJypF7J7qoqJ5JcuzXs
0,54087d1b-6dab-403b-93eb-c33162f10aae,jVF1elDY6pZ1zZfoWO9K
0,54087d1b-6dab-403b-93eb-c33162f10aae,k01MJnr4IWnpM9vUG9Cp
0,54087d1b-6dab-403b-93eb-c33162f10aae,n7nLaGcBeHJg54aNs7eb
0,54087d1b-6dab-403b-93eb-c33162f10aae,Jvrv1829TmalCfkvRg7z


In [10]:
# dropping ingredients column from recipe_df
recipe_df = recipe_df.drop(['ingredients'], axis=1)
recipe_df.head()

,id,name,cuisine,diet,description,meal_type,image_url,cooking_time,prep_time
0,54087d1b-6dab-403b-93eb-c33162f10aae,Zucchini Curry,[Indian],[Vegan],Zucchini curry is a vegetarian dish that featu...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,66.0,0.0
1,b7f22d40-8a95-4877-8d50-0f493c61738a,Gujarati Kadhi,"[Indian, Gujarati]",[Veg],Gujarati kadhi is a sweet tangy yogurt-based d...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,36.0,0.0
2,a15df9da-6efd-4bf1-ba6e-0f5fa0fc6a4e,Sabudana Khichdi,[Indian],[Veg],Sabudana Khichdi is a popular Indian dish made...,[Breakfast],https://res.cloudinary.com/dgakjrw8j/image/upl...,58.0,240.0
3,f5e954c2-821a-4541-a3b0-d126baa02ac5,Beetroot Palya,"[Indian, South Indian]",[Vegan],"This recipe is prepared using beetroot, grated...",[Main Course Dry],https://res.cloudinary.com/dgakjrw8j/image/upl...,31.0,0.0
4,f7b53d60-5c24-4512-bba5-90af1f27f8ce,Jain Red Sauce Pasta,[Italian],[Veg],Red sauce pasta on autopilot. Lightly flavoure...,[One Pot Meal],https://res.cloudinary.com/dgakjrw8j/image/upl...,46.0,0.0


In [11]:
ingredients_df = pd.json_normalize(recipe_ingredients_dict['data']['Ingredients_DB'])
ingredients_df.head()

,displayName,allergens,_id
0,Rice Cakes,[],78a1dd37-5ea4-473d-bea9-60e73fa74af9
1,Fish Cake,[],4c3c805b-6169-4e84-b2c0-67dee660e97e
2,new ingredient,[],55c89473-b57f-4394-b65b-f7edd817ffd8
3,Shredded Cabbage,[],9b5533eb-46b8-4ad7-8296-ea4ee3619f53
4,Tomato Puree,[],a1d7c043-7dd9-449c-9f97-ab7b0a372f46


In [12]:
# Ingredients
ingredients_df = ingredients_df.drop(['allergens'], axis=1).rename(columns={'displayName': "name", "_id": "id"})
ingredients_df.head()

,name,id
0,Rice Cakes,78a1dd37-5ea4-473d-bea9-60e73fa74af9
1,Fish Cake,4c3c805b-6169-4e84-b2c0-67dee660e97e
2,new ingredient,55c89473-b57f-4394-b65b-f7edd817ffd8
3,Shredded Cabbage,9b5533eb-46b8-4ad7-8296-ea4ee3619f53
4,Tomato Puree,a1d7c043-7dd9-449c-9f97-ab7b0a372f46


In [13]:
# Adding ingredients names list and category names list in recipe_df
recipe_category_name_map = (
    category_recipe_map
    .merge(category_df, left_on='category_id', right_on='id', how='inner')
    .groupby('recipe_id', as_index=False)
    .agg(**{"category_names": ("category_name", list)})
)

recipe_ingredients_name_map = (
    recipe_ingredients_df
    .merge(ingredients_df, left_on='ingredients_id', right_on='id', how='inner')
    .groupby('recipe_id', as_index=False)
    .agg(**{"ingredients_names": ("name", list)})
)

recipe_df = (
    recipe_df
    .merge(recipe_category_name_map, left_on='id', right_on='recipe_id', how='left')
    .drop(['recipe_id'], axis=1)
    .merge(recipe_ingredients_name_map, left_on='id', right_on='recipe_id', how='left')
    .drop(['recipe_id'], axis=1)
    # filling with empty list wherever ingredients name or category names is missing
    .assign(
        category_names = lambda df_: df_['category_names'].map(lambda x: [] if pd.isnull(x) is True else x),
        ingredients_names = lambda df_: df_['ingredients_names'].map(lambda x: [] if pd.isnull(x) is True else x)
    )
)

recipe_df.head()

,id,name,cuisine,diet,description,meal_type,image_url,cooking_time,prep_time,category_names,ingredients_names
0,54087d1b-6dab-403b-93eb-c33162f10aae,Zucchini Curry,[Indian],[Vegan],Zucchini curry is a vegetarian dish that featu...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,66.0,0.0,"[Perfect With Chapati, Fusion Recipes]","[Zucchini, Refined Oil, Jeera, Onion, Garlic, ..."
1,b7f22d40-8a95-4877-8d50-0f493c61738a,Gujarati Kadhi,"[Indian, Gujarati]",[Veg],Gujarati kadhi is a sweet tangy yogurt-based d...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,36.0,0.0,"[Vegetarian Delights, Kem Cho Gujarat!]","[Mustard Seeds, Jeera, Hing, Dried Red Chilli,..."
2,a15df9da-6efd-4bf1-ba6e-0f5fa0fc6a4e,Sabudana Khichdi,[Indian],[Veg],Sabudana Khichdi is a popular Indian dish made...,[Breakfast],https://res.cloudinary.com/dgakjrw8j/image/upl...,58.0,240.0,[Breakfasts],"[Medium Tapioca Pearls, Water, Peanuts (Raw), ..."
3,f5e954c2-821a-4541-a3b0-d126baa02ac5,Beetroot Palya,"[Indian, South Indian]",[Vegan],"This recipe is prepared using beetroot, grated...",[Main Course Dry],https://res.cloudinary.com/dgakjrw8j/image/upl...,31.0,0.0,"[Homely Recipes, Side Dish Specialities]","[Refined Oil, Green Chilli, Jeera, Mustard See..."
4,f7b53d60-5c24-4512-bba5-90af1f27f8ce,Jain Red Sauce Pasta,[Italian],[Veg],Red sauce pasta on autopilot. Lightly flavoure...,[One Pot Meal],https://res.cloudinary.com/dgakjrw8j/image/upl...,46.0,0.0,[Jain Friendly Meals],"[Parsley, Green Chilli, Pomace Olive Oil, Toma..."


In [14]:
def make_recipe_document(row):
    # id	name	cuisine	diet	description	meal_type	image_url	cooking_time	prep_time	ingredients
    # print(row)
    name: str =  row['name']
    cuisine: list[str] = row['cuisine']
    diet: list[str] = row['diet']
    description: str = row['description']
    meal_type: list[str] = row['meal_type']
    category: list[str] = row['category_names']
    ingredients: list[str] = row['ingredients_names']
    document = (
        f"Recipe name is {name}.\n"
        f"{description}\n"
        f"Cuisine is {', '.join(cuisine)}.\n"
        f"Diet is {', '.join(diet)}.\n"
        f"Meal type is {', '.join(meal_type)}.\n"
        f"Food Categories are {', '.join(category)}.\n"
        f"Ingredients are {', '.join(ingredients)}."
    )
    return document

In [15]:
# creating document column for recipe
recipe_df['document'] = recipe_df.apply(make_recipe_document, axis=1)
recipe_df.head()

,id,name,cuisine,diet,description,meal_type,image_url,cooking_time,prep_time,category_names,ingredients_names,document
0,54087d1b-6dab-403b-93eb-c33162f10aae,Zucchini Curry,[Indian],[Vegan],Zucchini curry is a vegetarian dish that featu...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,66.0,0.0,"[Perfect With Chapati, Fusion Recipes]","[Zucchini, Refined Oil, Jeera, Onion, Garlic, ...",Recipe name is Zucchini Curry.\nZucchini curry...
1,b7f22d40-8a95-4877-8d50-0f493c61738a,Gujarati Kadhi,"[Indian, Gujarati]",[Veg],Gujarati kadhi is a sweet tangy yogurt-based d...,[Main Course Gravy],https://res.cloudinary.com/dgakjrw8j/image/upl...,36.0,0.0,"[Vegetarian Delights, Kem Cho Gujarat!]","[Mustard Seeds, Jeera, Hing, Dried Red Chilli,...",Recipe name is Gujarati Kadhi.\nGujarati kadhi...
2,a15df9da-6efd-4bf1-ba6e-0f5fa0fc6a4e,Sabudana Khichdi,[Indian],[Veg],Sabudana Khichdi is a popular Indian dish made...,[Breakfast],https://res.cloudinary.com/dgakjrw8j/image/upl...,58.0,240.0,[Breakfasts],"[Medium Tapioca Pearls, Water, Peanuts (Raw), ...",Recipe name is Sabudana Khichdi.\nSabudana Khi...
3,f5e954c2-821a-4541-a3b0-d126baa02ac5,Beetroot Palya,"[Indian, South Indian]",[Vegan],"This recipe is prepared using beetroot, grated...",[Main Course Dry],https://res.cloudinary.com/dgakjrw8j/image/upl...,31.0,0.0,"[Homely Recipes, Side Dish Specialities]","[Refined Oil, Green Chilli, Jeera, Mustard See...",Recipe name is Beetroot Palya.\nThis recipe is...
4,f7b53d60-5c24-4512-bba5-90af1f27f8ce,Jain Red Sauce Pasta,[Italian],[Veg],Red sauce pasta on autopilot. Lightly flavoure...,[One Pot Meal],https://res.cloudinary.com/dgakjrw8j/image/upl...,46.0,0.0,[Jain Friendly Meals],"[Parsley, Green Chilli, Pomace Olive Oil, Toma...",Recipe name is Jain Red Sauce Pasta .\nRed sau...


In [16]:
print(recipe_df['document'][0])

Recipe name is Zucchini Curry.
Zucchini curry is a vegetarian dish that features zucchini as the main ingredient. The curry typically includes a blend of spices and seasonings, such as turmeric, cumin, and coriander, along with garlic, onions, and tomatoes. The addition of coconut milk makes it creamy. Pair with Rice/Roti or Appam
Cuisine is Indian.
Diet is Vegan.
Meal type is Main Course Gravy.
Food Categories are Perfect With Chapati, Fusion Recipes.
Ingredients are Zucchini, Refined Oil, Jeera, Onion, Garlic, Green Chilli, Basil, Dried Basil, Salt, Coriander Powder, Kashmiri Mirch, Garam Masala, Tomato, Water, Coconut Milk,  Coriander Leaves.


In [17]:
recipe_df.to_parquet(constants.recipe_file)